# Deploy Huggingface Model to Serverless Sagemaker

- set the kernel to a pytorch kernel
- load a model from huggingface hub
- register the model

### Links

- https://huggingface.co/docs/sagemaker/inference#deploy-a-model-from-the-hub
- https://www.youtube.com/watch?v=l9QZuazbzWM


In [5]:
!pip install sagemaker --upgrade --quiet

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [6]:
import sagemaker
from sagemaker.huggingface.model import HuggingFaceModel

## 1. Using Sagemaker SDK
#### 1.1 Create a Model

In [14]:
# Hub model configuration <https://huggingface.co/models>
hub = {
  'HF_MODEL_ID':'distilbert-base-uncased-finetuned-sst-2-english', # model_id from hf.co/models
  'HF_TASK':'sentiment-analysis'                           # NLP task you want to use for predictions
}

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

huggingface_model = HuggingFaceModel(
   env=hub,                                                # configuration for loading model from Hub
   role=role,                                              # IAM role with permissions to create an endpoint
   transformers_version="4.6",                             # Transformers version used
   pytorch_version="1.7",                                  # PyTorch version used
   py_version='py36',                                      # Python version used
)

#### 1.2 Deploy a Model and call it.

In [23]:
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    serverless_inference_config=ServerlessInferenceConfig(memory_size_in_mb=1024, max_concurrency=5)
)

----!

In [26]:
%%time

data = {
   "inputs": "I really like this place!"
}
predictor.predict(data)

InternalDependencyException: An error occurred (InternalDependencyException) when calling the InvokeEndpoint operation: An exception occurred from internal dependency. Please contact customer support regarding request 3c9b841a-e48d-4374-9767-9f4835da3b43.

In [27]:
predictor.endpoint_name

'huggingface-pytorch-inference-2022-02-01-13-58-48-937'

#### 1.3 Remove the serverless sagemaker endpoint

In [67]:
!aws sagemaker delete-endpoint --endpoint-name $predictor.endpoint_name

## 2. Huggingface on Serverless Sagemaker deployed by Serverless.com

- Using sagemaker sdk you can deploy an endpoint, but you can only reach a sagemaker endpoint from within an AWS account.
- Typically you want to expose your model to the outside world, another network
- You want to deploy yoouor models from IAC.

#### 2.1 create a sagemaker moodel

In [13]:
# create a sagemaker model
# https://github.com/aws/sagemaker-python-sdk/blob/d635faff4ac54f80465f7bc7f3181f67336e249a/src/sagemaker/model.py#L261
# Maybe not the best way to create a sagemaker model, but i didn't found a better way.

huggingface_model._create_sagemaker_model(instance_type="ml.m5.xlarge", accelerator_type=None, tags=None)
sagemaker_model_name = huggingface_model.name
sagemaker_model_name

AttributeError: 'NoneType' object has no attribute 'boto_session'

#### 2.2 create a lambda to call the sagemaker endpoint

In [63]:
%%writefile lambda_handler.py
import json
import boto3
import os

runtime_client = boto3.client("runtime.sagemaker")
sagemaker_endpoint_name = os.environ["SAGEMAKER_ENDPOINT_NAME"]

def handler(event, context):

    print(f"making a prediction on the text: {event['body']}")
    
    response = runtime_client.invoke_endpoint(
        Body=event["body"],
        EndpointName=sagemaker_endpoint_name,
        Accept="application/json",
        ContentType="application/json",
    )
    
    print(f"prediction: {response}")

    return {
        'statusCode': 200,
        'body': json.dumps(prediction)
    }


Overwriting lambda_handler.py


#### 2.3 create a serverless.yml file

In [4]:
# function to write variables to a textfile
# https://github.com/ipython/ipython/issues/6701#issuecomment-382640776
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [3]:
%%writetemplate serverless.yml
service: huggingface-on-serverless-sagemaker

provider:
  name: aws
  region: eu-west-1 
  runtime: python3.8
  iam:
    role:
      managedPolicies: arn:aws:iam::aws:policy/AdministratorAccess


functions:
  huggingface:
    handler: lambda_handler.handler
    timeout: 120
    memorySize: 128 
    events:
      - http:
          path: prediction
          method: post
    environment:
      SAGEMAKER_ENDPOINT_NAME: !GetAtt SageMakerEndpoint.EndpointName

resources:
  Resources:
    SageMakerEndpointConfig:
      Type: AWS::SageMaker::EndpointConfig
      Properties:
        ProductionVariants:
          - ModelName: {sagemaker_model_name}
#             InitialInstanceCount: 1
#             InitialVariantWeight: 1.0
#             VariantName: SageMakerModel
            ServerlessConfig:
              MaxConcurrency: 1
              MemorySizeInMB: 1024

    SageMakerEndpoint:
      Type: AWS::SageMaker::Endpoint
      Properties:
        EndpointConfigName: !GetAtt SageMakerEndpointConfig.EndpointConfigName
        EndpointName: huggingface-serverless-sagemaker-endpoint


UsageError: Cell magic `%%writetemplate` not found.


#### 2.4 open an AWS cloud shell and deploy the application

```
git clone https://github.com/vincentclaes/huggingface-on-serverless-sagemaker.git

cd huggingface-on-serverless-sagemaker/

npm install serverless

/home/cloudshell-user/node_modules/serverless/bin/serverless.js deploy
```

#### 2.6 Call the Endpoint

In [71]:
%%time
!curl -d '{"inputs":"some very much wow positive text!"}' -H "Content-Type: application/json" -X POST  https://2kqraqs038.execute-api.eu-west-1.amazonaws.com/dev/prediction


{"message": "Endpoint request timed out"}CPU times: user 449 ms, sys: 111 ms, total: 560 ms
Wall time: 29.3 s


#### 2.5 remove the stack

```
/home/cloudshell-user/node_modules/serverless/bin/serverless.js remove
```

In [77]:
import boto3
data = '{"inputs":"some very much wow positive text!"}'

response = boto3.client("runtime.sagemaker").invoke_endpoint(
        Body=data,
#         EndpointName="huggingface-serverless-sagemaker-endpoint",
        EndpointName="huggingface-pytorch-inference-2022-02-01-12-24-58-432",
        Accept="application/json",
        ContentType="application/json",
    )

In [78]:
response

{'ResponseMetadata': {'RequestId': '559419df-0e70-42f7-8467-9ab9c092c6f7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '559419df-0e70-42f7-8467-9ab9c092c6f7',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Tue, 01 Feb 2022 12:31:27 GMT',
   'content-type': 'application/json',
   'content-length': '49'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7fb39abbe1d0>}